In [ ]:
# Exploration des bases de données enregistrées dans minio
# Create filesystem object
import os

import s3fs
import pandas
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "radjerad"
FILE_KEY_S3 = "inpi/inpi_2019.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3
with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df_bpe = pd.read_csv(file_in, sep=";")